In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datasets/Google.csv")
df.index = pd.DatetimeIndex(df['Date'].values)
close = df["Close"]

In [2]:
def mp_num_co_events(close_idx, t1, molecule):
    t1 = t1.fillna(close_idx[-1])
    t1 = t1[t1 >= molecule[0]]
    t1 = t1.loc[:t1[molecule].max()]
    
    iloc = close_index.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=close_idx[loc[0]: iloc[1] + 1])
    for t_in, t_out in t1.iterterms():
        count.loc[t_in: t_out] += 1
    return count.loc[molecule[0]: t1[molecule].max()]

In [3]:
def mp_sample_tw(t1, num_co_events, molecule):
    wght = pd.Series(index=molecule)
    for t_in, t_out in t1.loc[wght.index].iteritems():
        wght.loc[t_in] = (1. / num_co_events.loc[t_in: t_out]).mean()
    return wght

In [4]:
def get_ind_matrix(bar_idx, t1):
    ind_m = pd.DataFrame(0, index=bar_idx,
                         columns=range(t1.shape[0]))
    for  i, (t0_, t1_) in enumerate(t1.iteritems()):
        ind_m.loc[t0_:t1_, i] = 1
    return ind_m

def get_avg_uniquness(ind_m):
    c = ind_m.sum(axis=1)
    u = ind_m.div(c, axis=0)
    avg_u = u[u>0].mean()
    return avg_u

In [5]:
def seq_bootstrap(ind_m, s_length=None):
    if s_length is None:
        s_length = ind_m.shape[1]
    phi = []
    while len(phi) < s_length:
        avg_u = pd.Series()
        for i in ind_m:
            ind_m_ = ind_m[phi + [i]]
            avg_u.loc[i] = get_avg_uniquness(ind_m_).iloc[-1]
        prob = avg_u / avg_u.sum()
        phi += [np.random.choice(ind_m.columns, p=prob)]
    return phi

In [6]:
t1 = pd.Series([2, 3, 5], index=[0, 2, 4])
bar_idx = range(t1.max() + 1)

In [7]:
ind_m = get_ind_matrix(bar_idx, t1)
phi = np.random.choice(ind_m.columns, size=ind_m.shape[1])
print(phi)

[1 0 0]


In [8]:
get_avg_uniquness(ind_m[phi])

1    0.666667
0    0.444444
0    0.444444
dtype: float64

In [9]:
phi = seq_bootstrap(ind_m)

In [10]:
phi

[2, 0, 1]

In [11]:
get_avg_uniquness(ind_m[phi])

2    1.000000
0    0.833333
1    0.750000
dtype: float64

In [12]:
def get_rnd_t1(num_obs, num_bars, max_h):
    t1 = pd.Series()
    for i in xrange(num_obs):
        ix = np.random.randin(0, num_bars)
        val = ix + np.random.randint(1, max_h)
    return t1.sort_index()

In [13]:
def auxMC(num_obs, num_bars, max_h):
    t1 = get_rnd_t1(num_obs, num_bars, max_h)
    bar_idx = range(t1.max() + 1)
    ind_m = get_ind_matrix(bar_idx, t1)
    phi = np.random.choice(ind_m.columns, sizez=ind_m.shape[1])
    std_u = get_avg_uniquness(ind_m[phi]).mean()
    phi = seq_bootstrap(ind_m)
    seq_u = get_avg_uniquness(ind_m[phi]).mean()
    return {'std_u': std_u, 'seq_u': seq_u}

In [14]:
def mp_sample_w(t1, num_co_events, close, molecule):
    ret = np.log(close).diff()
    wght = pd.Series(index=molecule)
    for t_in, t_out in t1.loc[wght.index].iteritems():
        wght.loc[t_in] = (ret.loc[t_in:t_out] / num_co_events.loc[t_in:t_out]).sum()
    return wght.abs()

In [15]:
def get_time_decay(tw, clf_last_w=1.):
    clf_w = tw.sort_index().cumsum()
    if clf_last_w >= 0:
        slope = (1. - clf_last_w) / clf_w.iloc[-1]
    else:
        slope = 1. / ((clf_last_w + 1) * clf_w.iloc[-1])
    const = 1. - slope * clf_w.iloc[-1]
    clf_w = const + slope * clf_w
    clf_w[clf_w < 0] = 0
    print(const, slope)
    return clf_w